In [ ]:
# !git clone https://github.com/p-skaisgiris/temporal_advice_rm_noisy.git > /dev/null
# !pip install -r temporal_advice_rm_noisy/requirements.txt > /dev/null

In [ ]:
%cd temporal_advice_rm_noisy/
!echo $PYTHONPATH
%env PYTHONPATH="$/env/python:src:src/envs"
!echo $PYTHONPATH

In [ ]:
# Train agent without a reward machine
!python scripts/train_agent.py --env=Kitchen-v2  --rm-update-algo=no_rm --seed=123 --frames=1000000 --epochs=8 --batch-size=2048 --frames-per-proc=32768 --entropy-coef=0.01 --recurrence=1 --detector-recurrence=1 --detector-epochs=8 --detector-batch-size=2048

In [ ]:
# Visualize the learned policy network without RM
python scripts/minigrid_visualize.py --env=Kitchen-v2 --model=storage/no_rm-Kitchen-v2-seed123/train/ --rm-update-algo=no_rm --gif=kitchen-no-rm --seed=123 --no-render

In [ ]:
# Collect training data for the detector
!python scripts/collect_offline_data.py --env=Kitchen-v2 --seed=111 --episodes=2000
%mv collect_Kitchen-v2.pt train_Kitchen-v2.pt
!python scripts/collect_offline_data.py --env=Kitchen-v2 --seed=222 --episodes=100
%mv collect_Kitchen-v2.pt test_Kitchen-v2.pt

In [ ]:
# Pre-train detector
!python scripts/pretrain_detector.py --env=Kitchen-v2 --train-data-path=train_Kitchen-v2.pt --test-data-path=test_Kitchen-v2.pt --use-mem

In [ ]:
# Train agent with tdm and a pretrained detector
!python scripts/train_agent.py --env=Kitchen-v2  --rm-update-algo=tdm --seed=123 --frames=1000000 --epochs=8 --batch-size=2048 --frames-per-proc=32768 --entropy-coef=0.01 --recurrence=4 --detector-model=Kitchen-v2_tdm_pretrain_seed1/best --detector-recurrence=4 --detector-epochs=8 --detector-batch-size=2048

In [ ]:
# Run the learned policy network with pretrained detector TDM and reward machine
python scripts/minigrid_visualize.py --env=Kitchen-v2 --model=storage/tdm-r-Kitchen-v2-seed123/train/ --rm-update-algo=tdm --gif=kitchen-tdm-pretrained --seed=123 --use-mem --use-mem-detector --no-render